In [2]:
from skfp.fingerprints import AtomPairFingerprint

smiles_list = ["O=S(=O)(O)CCS(=O)(=O)O", "O=C(O)c1ccccc1O"]

atom_pair_fingerprint = AtomPairFingerprint()

X = atom_pair_fingerprint.transform(smiles_list)
print(X)
print(X.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2, 2048)


In [23]:
from skfp.preprocessing import ConformerGenerator, MolFromSmilesTransformer
from skfp.fingerprints import WHIMFingerprint

smiles_list = ["O=S(=O)(O)CCS(=O)(=O)O", "O=C(O)c1ccccc1O"]

mol_from_smiles = MolFromSmilesTransformer()
conf_gen = ConformerGenerator()
fp = WHIMFingerprint()
print(fp.requires_conformers)  # True

mols_list = mol_from_smiles.transform(smiles_list)
mols_list = conf_gen.transform(mols_list)

X = fp.transform(mols_list)
print(X)


True
[[4.3190e+00 1.1930e+00 6.3200e-01 7.0300e-01 1.9400e-01 2.0000e-01
  2.0000e-01 2.0000e-01 5.8300e-01 5.5600e-01 3.3000e-01 5.2330e+00
  6.6900e-01 5.3100e-01 8.1300e-01 1.0400e-01 2.0000e-01 2.0000e-01
  2.0000e-01 9.2700e-01 1.2300e-01 2.2800e-01 4.4720e+00 8.5600e-01
  5.4000e-01 7.6200e-01 1.4600e-01 2.0000e-01 2.0000e-01 2.0000e-01
  6.5800e-01 2.3500e-01 2.5100e-01 4.6300e+00 1.1790e+00 6.7000e-01
  7.1500e-01 1.8200e-01 2.0000e-01 2.0000e-01 2.0500e-01 6.7900e-01
  5.3900e-01 3.7800e-01 3.9780e+00 8.2700e-01 4.2700e-01 7.6000e-01
  1.5800e-01 2.0000e-01 2.0000e-01 2.0000e-01 5.1100e-01 2.4600e-01
  1.5800e-01 4.4130e+00 1.2420e+00 6.6500e-01 6.9800e-01 1.9700e-01
  2.0000e-01 2.0000e-01 2.0000e-01 6.0700e-01 6.0000e-01 3.6400e-01
  6.0720e+00 1.1170e+00 8.9100e-01 7.5200e-01 1.3800e-01 2.0000e-01
  2.0000e-01 2.0000e-01 1.2410e+00 3.4500e-01 6.7200e-01 6.1450e+00
  6.4330e+00 5.8680e+00 6.4790e+00 5.2320e+00 6.3210e+00 8.0790e+00
  8.6400e+00 6.6380e+00 6.7050e+00 9.3490e+

In [24]:
from skfp.datasets.moleculenet import load_clintox
from skfp.metrics import multioutput_auroc_score
from skfp.model_selection import scaffold_train_test_split
from skfp.fingerprints import ECFPFingerprint, MACCSFingerprint
from skfp.preprocessing import MolFromSmilesTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, make_union

smiles, y = load_clintox()
smiles_train, smiles_test, y_train, y_test = scaffold_train_test_split(
    smiles, y, test_size=0.2
)

pipeline = make_pipeline(
    MolFromSmilesTransformer(),
    make_union(ECFPFingerprint(count=True), MACCSFingerprint()),
    RandomForestClassifier(random_state=0),
)
pipeline.fit(smiles_train, y_train)

y_pred_proba = pipeline.predict_proba(smiles_test)
auroc = multioutput_auroc_score(y_test, y_pred_proba)
print(f"AUROC: {auroc:.2%}")

ValueError: Both true labels and predictions must have the same shape, got: true labels 2, predictions (2, 296, 2)

In [25]:
from sklearn . ensemble import RandomForestClassifier
from sklearn . pipeline import make_pipeline
from skfp . fingerprints import ECFPFingerprint


pipeline = make_pipeline (
    ECFPFingerprint ( n_jobs = -1) ,
    RandomForestClassifier( n_jobs = -1 , random_state =0)
)

pipeline.fit( smiles_train , y_train )
y_pred = pipeline.predict( smiles_test )

In [26]:
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader

dataset_names = ["ogbg-molhiv", "ogbg-molpcba", "ogbg-ppa", "ogbg-code"]

dataset = PygGraphPropPredDataset(name = dataset_names[0]) 

split_idx = dataset.get_idx_split() 
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

ImportError: cannot import name 'PygGraphPropPredDataset' from 'ogb.graphproppred' (/home/adrian/anaconda3/envs/um-labs/lib/python3.12/site-packages/ogb/graphproppred/__init__.py)

In [16]:
from skfp.datasets.moleculenet import load_ogb_splits

train, valid, test  = load_ogb_splits('ClinTox')

print("train: ", len(train))
print("valid: ", len(valid))  
print("test: ", len(test))


train:  1181
valid:  148
test:  148


In [17]:
from skfp.datasets.moleculenet import load_bace
from skfp.fingerprints import ECFPFingerprint
from skfp.model_selection import FingerprintEstimatorGridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

smiles, labels = load_bace()

fp = ECFPFingerprint(n_jobs=-1)
fp_params = {"radius": [2, 3]}

clf = RandomForestClassifier(n_jobs=-1)
clf_params = {"min_samples_split": [2, 3, 4]}
clf_cv = GridSearchCV(clf, clf_params)

fp_cv = FingerprintEstimatorGridSearch(fp, fp_params, clf_cv)
fp_cv.fit(smiles, labels)    


FingerprintEstimatorGridSearch(estimator_cv=GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),
                                                         param_grid={'min_samples_split': [2,
                                                                                           3,
                                                                                           4]}),
                               fingerprint=ECFPFingerprint(n_jobs=-1),
                               fp_param_grid={'radius': [2, 3]})

In [18]:
from skfp.preprocessing import BMSFilter
smiles = ["C", "O", "c1(C=O)cc(OC)c(O)cc1"]
filt = BMSFilter()

filtered_mols = filt.transform(smiles)
filtered_mols

['C', 'O']

In [4]:
from skfp.preprocessing import RuleOfThree
smiles = ['C=CCNC(=S)NCc1ccccc1OC', 'C=CCOc1ccc(Br)cc1/C=N/O', 'C=CCNc1ncnc2ccccc12']
filt = RuleOfThree()

filtered_mols = filt.transform(smiles)
print(filtered_mols)

filt = RuleOfThree(extended=True)
filtered_mols = filt.transform(smiles)
print(filtered_mols)

['C=CCNC(=S)NCc1ccccc1OC', 'C=CCOc1ccc(Br)cc1/C=N/O', 'C=CCNc1ncnc2ccccc12']
['C=CCNc1ncnc2ccccc12']


<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterMatcherBase> already registered; second conversion method ignored.


In [4]:
from skfp.filters import BMSFilter
smiles = ["C", "O", "c1(C=O)cc(OC)c(O)cc1"]
filt = BMSFilter()
filtered_mols = filt.transform(smiles)
filtered_mols

['C', 'O']

In [3]:
from skfp.filters import MLSMRFilter
smiles = ["C", "O", "OP(=O)(O)O"]
filt = MLSMRFilter()

filtered_mols = filt.transform(smiles)
filtered_mols

['C', 'O']

In [5]:
from typing import Optional

from rdkit.Chem import FilterCatalog, Mol
from rdkit.Chem.rdfiltercatalog import FilterCatalogParams

from skfp.bases.base_filter import BaseFilter


class MLSMRFilter2(BaseFilter):
    """
    MLSMR filter.

    NIH Molecular Libraries Small Molecule Repository (MLSMR) "Excluded Functionality
    Filters". MLSMR was a library of diverse small molecules (now discontinued).

    Filter origin and rules rationale are currently unknown, and there seems to be no
    literature reference. Rule definitions are available in the RDKit code [1]_.

    Parameters
    ----------
    allow_one_violation : bool, default=False
        Whether to allow violating one of the rules for a molecule. This makes the
        filter less restrictive.

    return_indicators : bool, default=False
        Whether to return a binary vector with indicators which molecules pass the
        filter, instead of list of molecules.

    n_jobs : int, default=None
        The number of jobs to run in parallel. :meth:`transform_x_y` and
        :meth:`transform` are parallelized over the input molecules. ``None`` means 1
        unless in a :obj:`joblib.parallel_backend` context. ``-1`` means using all
        processors. See Scikit-learn documentation on ``n_jobs`` for more details.

    batch_size : int, default=None
        Number of inputs processed in each batch. ``None`` divides input data into
        equal-sized parts, as many as ``n_jobs``.

    verbose : int, default=0
        Controls the verbosity when filtering molecules.

    References
    ----------
    .. [1] `RDKit MLSMR filter definitions
        <https://github.com/rdkit/rdkit/blob/e4f4644a89d6446ddebda0bf396fa4335324c41c/Code/GraphMol/FilterCatalog/chembl_mlsmr.in>`_

    Examples
    --------
    >>> from skfp.filters import MLSMRFilter
    >>> smiles = ["C", "O", "OP(=O)(O)O"]
    >>> filt = MLSMRFilter()
    >>> filt
    MLSMRFilter()

    >>> filtered_mols = filt.transform(smiles)
    >>> filtered_mols
    ['C', 'O']
    """

    def __init__(
        self,
        allow_one_violation: bool = False,
        return_indicators: bool = False,
        n_jobs: Optional[int] = None,
        batch_size: Optional[int] = None,
        verbose: int = 0,
    ):
        super().__init__(
            allow_one_violation=allow_one_violation,
            return_indicators=return_indicators,
            n_jobs=n_jobs,
            batch_size=batch_size,
            verbose=verbose,
        )
        self._filters = self._load_filters()

    def _load_filters(self) -> FilterCatalog:
        filter_rules = FilterCatalogParams.FilterCatalogs.CHEMBL_MLSMR
        params = FilterCatalog.FilterCatalogParams()
        params.AddCatalog(filter_rules)
        filters = FilterCatalog.FilterCatalog(params)
        return filters

    def _apply_mol_filter(self, mol: Mol) -> bool:
        errors = len(self._filters.GetMatches(mol))
        return not errors or (self.allow_one_violation and errors == 1)

In [6]:
smiles = ["C", "O", "OP(=O)(O)O"]
filt = MLSMRFilter2()

filtered_mols = filt.transform(smiles)
filtered_mols

['C', 'O']

In [27]:
from dundee import DundeeFilter
from tests_dundee import test_dundee, test_dundee_parallel, test_dundee_allowing_one_violation

In [28]:
smiles = ["C", "O", "OP(=O)(O)O"]
filt = DundeeFilter(allow_one_violation=True)

filtered_mols = filt.transform(smiles)
filtered_mols

['C', 'O', 'OP(=O)(O)O']

In [30]:
smiles = ["C", "O", "OP(=O)(O)O"]
# test_dundee(smiles)
test_dundee_parallel(smiles)
test_dundee_allowing_one_violation(smiles)

In [ ]:
smiles = ['C=CCNC(=S)NCc1ccccc1OC', 'C=CCOc1ccc(Br)cc1/C=N/O', 'C=CCNc1ncnc2ccccc12']

DundeeFilter(allow_one_violation=True)